In [ ]:
# PASTE THIS TO THE FIRST CELL OF THE NOTEBOOK IN ORDER TO HAVE WORKING IMPORTS
import sys
import os
current_dir = os.getcwd()
print(f"Current dir is {current_dir}")
parent_parent_dir = os.path.abspath(os.path.join(current_dir, '../../..')) # tweak so that you get dir of code project

sys.path.append(parent_parent_dir)
print(f"Added {parent_parent_dir} to sys.path")

In [ ]:
from sklearn.cluster import OPTICS
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt

## data import

In [ ]:
from src.features.get_x_y_tuples import get_x_y_tuple_list
from src.features.get_first_and_last_x_y_coordinates import get_first_and_last_x_y_coordinates 
from src.features.get_first_x_y_coordinates import get_first_x_y_coordinates
from src.features.get_last_x_y_coordinates import get_last_x_y_coordinates
def import_data(df_path):
    df_cuid = pd.read_csv(df_path)
    df_cuid_grouped_path = df_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    # 1.1 CONVERT FEATURES TO NUMBERS
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
    list_x_y_tuples = get_x_y_tuple_list(df_cuid_grouped, ['x','y'])
    first_last_x_coords, first_last_y_coords = get_first_and_last_x_y_coordinates(list_x_y_tuples)
    X = np.array([first_last_x_coords, first_last_y_coords]).T

    first_x_coords, first_y_coords = get_first_x_y_coordinates(list_x_y_tuples)
    first_coordinates = np.array([first_x_coords, first_y_coords]).T

    last_x_coords, last_y_coords = get_last_x_y_coordinates(list_x_y_tuples)
    last_coordinates = np.array([last_x_coords, last_y_coords]).T
    return df_cuid, df_cuid_grouped, X, first_coordinates, last_coordinates, list_x_y_tuples
        

In [ ]:
df_path_k729_2022 = f'{parent_parent_dir}/data/processed/k729_2022_cuid.csv'
df_path_k733_2020 = f'{parent_parent_dir}/data/processed/k733_2020_cuid.csv'
df_path_k733_2018 = f'{parent_parent_dir}/data/processed/k733_2018_cuid.csv'

df_cuid_k729_2022, df_cuid_grouped_k729_2022, X_k729_2022, first_coordinates_k729_2022, last_coordinates_k729_2022, list_x_y_tuples_k729_2022 = import_data(df_path_k729_2022)
df_cuid_k733_2020, df_cuid_grouped_k733_2020, X_k733_2020, first_coordinates_k733_2020, last_coordinates_k733_2020, list_x_y_tuples_k733_2020 = import_data(df_path_k733_2020)
df_cuid_k733_2018, df_cuid_grouped_k733_2018, X_k733_2018, first_coordinates_k733_2018, last_coordinates_k733_2018, list_x_y_tuples_k733_2018 = import_data(df_path_k733_2018)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Function to interpolate a list of x-y coordinates to a fixed number of points
def interpolate_path(x, y, num_points=100):
    # Calculate the cumulative distances between points
    distances = np.cumsum(np.sqrt(np.diff(x)**2 + np.diff(y)**2))
    distances = np.insert(distances, 0, 0)  # Insert 0 at the beginning (distance for the first point)

    # Create a uniform set of distances (from 0 to total length) to interpolate to
    uniform_distances = np.linspace(0, distances[-1], num_points)

    # Interpolate x and y coordinates at these uniform distances
    x_interpolated = np.interp(uniform_distances, distances, x)
    y_interpolated = np.interp(uniform_distances, distances, y)

    return x_interpolated, y_interpolated


In [ ]:
num_points_interpolation = 40
def get_interpolated_df_grouped(df_grouped):
    interpolated_paths = []
    x = df_grouped['x']
    y = df_grouped['y']
    for i in range(len(x)):
        x_interpolated, y_interpolated = interpolate_path(x[i], y[i], num_points=num_points_interpolation)
        interpolated_paths.append({
        'x': x_interpolated,
        'y': y_interpolated
    })
    interpolated_df = pd.DataFrame(interpolated_paths)
    return interpolated_df

In [ ]:
df_interpolated_k729_2022 = get_interpolated_df_grouped(df_cuid_grouped_k729_2022)
df_interpolated_k733_2020 = get_interpolated_df_grouped(df_cuid_grouped_k733_2020)
df_interpolated_k733_2018 = get_interpolated_df_grouped(df_cuid_grouped_k733_2018)

In [ ]:
# insert indices into dataframe for track_id column
df_interpolated_k729_2022.insert(0, 'track_id', df_cuid_grouped_k729_2022['track_id'])
df_interpolated_k733_2020.insert(0, 'track_id', df_cuid_grouped_k733_2020['track_id'])
df_interpolated_k733_2018.insert(0, 'track_id', df_cuid_grouped_k733_2018['track_id'])

In [ ]:
%matplotlib qt
from src.visualization.plot_vehicle_tracks_in_notebook import plot_vehicle_tracks_in_notebook
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k729_2022, df_interpolated_k729_2022, 'k729_2022 interpolated vehicle paths', linestyle='-', alpha=0.1)
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k729_2022, df_cuid_grouped_k729_2022, 'k729_2022 interpolated vehicle paths', linestyle=':', color='gray', alpha=0.2)

In [ ]:
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k733_2020, df_interpolated_k733_2020, 'k733_2020 interpolated vehicle paths', linestyle='-', alpha=0.1)
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k733_2020, df_cuid_grouped_k733_2020, 'k733_2020 interpolated vehicle paths', linestyle=':', color='gray', alpha=0.2)

In [ ]:
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k733_2018, df_interpolated_k733_2018, 'k733_2018 interpolated vehicle paths', linestyle='-', alpha=0.1)
plot_vehicle_tracks_in_notebook(plt.gca(), df_cuid_k733_2018, df_cuid_grouped_k733_2018, 'k733_2018 interpolated vehicle paths', linestyle=':', color='gray', alpha=0.2)

In [ ]:
from src.features.calculate_path_length import calculate_path_length
path_lengths_k729_2022 = calculate_path_length(list_x_y_tuples_k729_2022)
path_lengths_k733_2020 = calculate_path_length(list_x_y_tuples_k733_2020)
path_lengths_k733_2018 = calculate_path_length(list_x_y_tuples_k733_2018)

In [ ]:
path_lengths_k729_2022 = [[len]*num_points_interpolation for len in path_lengths_k729_2022]
path_lengths_k733_2020 = [[len]*num_points_interpolation for len in path_lengths_k733_2020]
path_lengths_k733_2018 = [[len]*num_points_interpolation for len in path_lengths_k733_2018]

In [ ]:
df_interpolated_k729_2022['path_length'] = path_lengths_k729_2022
df_interpolated_k733_2020['path_length'] = path_lengths_k733_2020
df_interpolated_k733_2018['path_length'] = path_lengths_k733_2018

In [ ]:
x = [val for sublist in df_interpolated_k729_2022['x'] for val in sublist]
y = [val for sublist in df_interpolated_k729_2022['y'] for val in sublist]
length = [val for sublist in df_interpolated_k729_2022['path_length'] for val in sublist]
X_k729_2022 = np.array([x, y, length]).T

x = [val for sublist in df_interpolated_k733_2020['x'] for val in sublist]
y = [val for sublist in df_interpolated_k733_2020['y'] for val in sublist]
length = [val for sublist in df_interpolated_k733_2020['path_length'] for val in sublist]
X_k733_2020 = np.array([x, y, length]).T

x = [val for sublist in df_interpolated_k733_2018['x'] for val in sublist]
y = [val for sublist in df_interpolated_k733_2018['y'] for val in sublist]
length = [val for sublist in df_interpolated_k733_2018['path_length'] for val in sublist]
X_k733_2018 = np.array([x, y, length]).T

In [ ]:

import json
with open(f'{parent_parent_dir}/src/models/optics/optimized_optics_clustering_parameters/optimization_parameters_clustering_start_end_report_notebook.json') as f:
        optics_params = json.load(f)


In [ ]:
max_eps_range = np.arange(1, 10, 1)
min_samples_range = np.arange(5, 10, 1)
metrics = [
    #'cosine', 
    'euclidean', 
    'manhattan', 
    #'braycurtis', 
    # 'canberra', 
    'chebyshev', 
    #'correlation', 
    'mahalanobis', 
    'minkowski', 
    #'seuclidean', 
    #'sqeuclidean'
]
# xis = np.arange(0.01, 0.1, 0.01)
# cluster_methods = ['dbscan', 'xi']
xis = np.arange(0, 0.1, 0.1)
cluster_methods = ['dbscan']

#### k729_2022

##### optics

In [ ]:
from src.models.optics.optimize_optics_parallelized_no_aic_bic import optimize_optics_parallelized_no_aic_bic
# optimization_results_k729_2022 = optimize_optics_parallelized_no_aic_bic(X=X_k729_2022, 
#                                                                          max_eps_range=optics_params['max_eps_range'], 
#                                                                          min_samples_range=optics_params['min_samples_range'], 
#                                                                          metrics=optics_params['metrics'],
#                                                                          cluster_methods=optics_params['cluster_methods'],
#                                                                          xis=optics_params['xis'])
# optimization_results_k729_2022 = optimize_optics_parallelized_no_aic_bic(X=X_k729_2022, 
#                                                                          max_eps_range=max_eps_range, 
#                                                                          min_samples_range=min_samples_range, 
#                                                                          metrics=metrics,
#                                                                          cluster_methods=cluster_methods,
#                                                                          xis=xis)

optimization_results_k729_2022 = {'silhouette': {'score': 0.3233328226747606,
  'epsilon': 6,
  'min_samples': 5,
  'metric': 'chebyshev',
  'cluster_method': 'dbscan',
  'xi': None},
 'calinski_harabasz': {'score': 899.1044953202556,
  'epsilon': 6,
  'min_samples': 5,
  'metric': 'chebyshev',
  'cluster_method': 'dbscan',
  'xi': None},
 'davies_bouldin': {'score': 0.5555195559992916,
  'epsilon': 5,
  'min_samples': 5,
  'metric': 'chebyshev',
  'cluster_method': 'dbscan',
  'xi': None}}

In [ ]:
from sklearn.cluster import OPTICS
optics_optimized_silhouette_k729_2022 = OPTICS(min_samples=optimization_results_k729_2022['silhouette']['min_samples'],
                                               max_eps=optimization_results_k729_2022['silhouette']['epsilon'],
                                               metric=optimization_results_k729_2022['silhouette']['metric'],
                                               cluster_method=optimization_results_k729_2022['silhouette']['cluster_method']

                                               ).fit(X_k729_2022)
optics_optimized_silhouette_k729_2022

In [ ]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D  # Import the 3D plotting tools

labels_silhouette_k729_2022 = optics_optimized_silhouette_k729_2022.labels_
unique_labels = set(labels_silhouette_k729_2022)
colors = plt.cm.get_cmap('viridis', len(unique_labels)) 

fig_optics_k729_2022 = plt.figure()
ax = fig_optics_k729_2022.add_subplot(111, projection='3d')

# Plot each cluster with a unique color
for label in unique_labels:
    if label == -1:
        # Noise points (label == -1)
        color = 'gray'  # Color for noise points (black, transparent)
    else:
        color = colors(label)  # Assign a color from the colormap

    # Select points belonging to the current label
    cluster_points = X_k729_2022[labels_silhouette_k729_2022 == label]

    # Plot the points in the cluster
    ax.plot(X_k729_2022[:,0], X_k729_2022[:,1], zs=0, zdir='z', color='gray', alpha=0.6)
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:,2], c=[color], label=f'Cluster {label}')

# Step 4: Add legend and titles
ax.set_title('OPTICS Clustering of Vehicle Paths')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.legend(loc='best')

plt.show()

##### svm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM
from mpl_toolkits.mplot3d import Axes3D  # Import the 3D plotting tools

# Step 2: Apply One-Class SVM for outlier detection
svm_model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)  # You can adjust the nu parameter for controlling the outlier ratio
svm_model.fit(X_k729_2022)

# Step 3: Get the labels (-1 for outliers, 1 for inliers)
labels = svm_model.predict(X_k729_2022)

In [ ]:
%matplotlib qt

fig_svm_k729_2022 = plt.figure()
ax = fig_svm_k729_2022.add_subplot(111, projection='3d')

# Use 'viridis' colormap for inliers and a specific color for outliers
colors = plt.cm.viridis(np.linspace(0, 1, len(X_k729_2022)))

# Plot background gray lines for all paths
ax.plot(X_k729_2022[:, 0], X_k729_2022[:, 1], zs=0, zdir='z', color='gray', alpha=0.6)

# Collect inliers and outliers for later plotting
inliers = X_k729_2022[labels == 1]   # Inliers
outliers = X_k729_2022[labels == -1] # Outliers

# Plot inliers (green)
ax.scatter(inliers[:, 0], inliers[:, 1], inliers[:, 2], c='green', label='Inlier')

# Plot outliers (red)
ax.scatter(outliers[:, 0], outliers[:, 1], outliers[:, 2], c='red', label='Outlier')

# Add legend and titles
ax.set_title('One-Class SVM Outlier Detection (3D)')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate (Path Length)')
ax.legend(loc='best')

plt.show()


##### pca k means

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

# Step 2: Apply PCA for dimensionality reduction
pca = PCA(n_components=2)  # Reduce to 2 dimensions for visualization
X_k729_2022_pca = pca.fit_transform(X_k729_2022)

# Step 3: Apply KMeans Clustering on PCA-reduced data
kmeans = KMeans(n_clusters=3)  # Specify the number of clusters
kmeans.fit(X_k729_2022_pca)
labels = kmeans.labels_

fig_pca_k729_2022 = plt.figure()
ax = fig_pca_k729_2022.add_subplot(111, projection='3d')

# Use 'viridis' colormap for clusters
colors = plt.cm.viridis(np.linspace(0, 1, len(set(labels))))

for label in set(labels):
    cluster_points = X_k729_2022_pca[labels == label]
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], color=colors[label], label=f'Cluster {label}')

# Add legend and titles
ax.set_title('PCA + KMeans Clustering')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.legend(loc='best')

plt.show()


##### pca optics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

# Step 2: Apply PCA for dimensionality reduction
pca = PCA(n_components=2)  # Reduce to 2 dimensions for visualization
X_k729_2022_pca = pca.fit_transform(X_k729_2022)

# Step 3: Apply KMeans Clustering on PCA-reduced data
optics = OPTICS(min_samples=5, max_eps=3, metric='chebyshev', cluster_method='dbscan')  # Reduce to 2 dimensions for visualization
optics.fit(X_k729_2022_pca)
labels = optics.labels_

fig_pca_optics_k729_2022 = plt.figure()
ax = fig_pca_optics_k729_2022.add_subplot(111, projection='3d')

# Use 'viridis' colormap for clusters
colors = plt.cm.viridis(np.linspace(0, 1, len(set(labels))))

for label in set(labels):
    cluster_points = X_k729_2022[labels == label]
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], color=colors[label], label=f'Cluster {label}')

# Add legend and titles
ax.set_title('PCA + KMeans Clustering')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.legend(loc='best')

plt.show()


##### pca dbscan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

# Step 2: Apply PCA for dimensionality reduction
pca = PCA(n_components=2)  # Reduce to 2 dimensions for visualization
X_k729_2022_pca = pca.fit_transform(X_k729_2022)

# Step 3: Apply KMeans Clustering on PCA-reduced data
dbscan_model = OPTICS(min_samples=5, max_eps=6, metric='chebyshev', cluster_method='dbscan')  # Reduce to 2 dimensions for visualization
optics.fit(X_k729_2022)
labels = optics.labels_

fig_pca_optics_k729_2022 = plt.figure()
ax = fig_pca_optics_k729_2022.add_subplot(111, projection='3d')

# Use 'viridis' colormap for clusters
colors = plt.cm.viridis(np.linspace(0, 1, len(set(labels))))

for label in set(labels):
    cluster_points = X_k729_2022[labels == label]
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], color=colors[label], label=f'Cluster {label}')

# Add legend and titles
ax.set_title('PCA + OPTICS Clustering')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.legend(loc='best')

plt.show()


#### k733_2020

##### optics

In [ ]:
from src.models.optics.optimize_optics_parallelized_no_aic_bic import optimize_optics_parallelized_no_aic_bic

optimization_results_k733_2020 = optimize_optics_parallelized_no_aic_bic(X=X_k733_2020, 
                                                                         max_eps_range=max_eps_range, 
                                                                         min_samples_range=min_samples_range, 
                                                                         metrics=metrics,
                                                                         cluster_methods=cluster_methods,
                                                                         xis=xis)
optimization_results_k733_2020

In [ ]:
from scipy import linalg
from sklearn.cluster import OPTICS
optics_optimized_silhouette_k733_2020 = OPTICS(min_samples=optimization_results_k733_2020['silhouette']['min_samples'],
                                               max_eps=optimization_results_k733_2020['silhouette']['epsilon'],
                                               metric=optimization_results_k733_2020['silhouette']['metric'],
                                               cluster_method=optimization_results_k733_2020['silhouette']['cluster_method'],
                                                metric_params={'VI': np.linalg.inv(np.cov(X_k733_2020.T))}
                                               ).fit(X_k733_2020)
optics_optimized_silhouette_k733_2020

In [ ]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D  # Import the 3D plotting tools

labels_silhouette_k733_2020 = optics_optimized_silhouette_k733_2020.labels_
unique_labels = set(labels_silhouette_k733_2020)
colors = plt.cm.get_cmap('viridis', len(unique_labels)) 

fig_optics_k733_2020 = plt.figure()
ax = fig_optics_k733_2020.add_subplot(111, projection='3d')

# Plot each cluster with a unique color
for label in unique_labels:
    if label == -1:
        # Noise points (label == -1)
        color = 'gray'  # Color for noise points (black, transparent)
    else:
        color = colors(label)  # Assign a color from the colormap

    # Select points belonging to the current label
    cluster_points = X_k733_2020[labels_silhouette_k733_2020 == label]

    # Plot the points in the cluster
    ax.plot(X_k733_2020[:,0], X_k733_2020[:,1], zs=0, zdir='z', color='gray', alpha=0.6)
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:,2], c=[color], label=f'Cluster {label}')

# Step 4: Add legend and titles
ax.set_title('OPTICS Clustering of Vehicle Paths')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.legend(loc='best')

plt.show()

##### svm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM
from mpl_toolkits.mplot3d import Axes3D  # Import the 3D plotting tools

# Step 2: Apply One-Class SVM for outlier detection
svm_model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)  # You can adjust the nu parameter for controlling the outlier ratio
svm_model.fit(X_k733_2020)

# Step 3: Get the labels (-1 for outliers, 1 for inliers)
labels = svm_model.predict(X_k733_2020)

In [ ]:
%matplotlib qt

fig_svm_k733_2020 = plt.figure()
ax = fig_svm_k733_2020.add_subplot(111, projection='3d')

# Use 'viridis' colormap for inliers and a specific color for outliers
colors = plt.cm.viridis(np.linspace(0, 1, len(X_k733_2020)))

# Plot background gray lines for all paths
ax.plot(X_k733_2020[:, 0], X_k733_2020[:, 1], zs=0, zdir='z', color='gray', alpha=0.6)

# Collect inliers and outliers for later plotting
inliers = X_k733_2020[labels == 1]   # Inliers
outliers = X_k733_2020[labels == -1] # Outliers

# Plot inliers (green)
ax.scatter(inliers[:, 0], inliers[:, 1], inliers[:, 2], c='green', label='Inlier')

# Plot outliers (red)
ax.scatter(outliers[:, 0], outliers[:, 1], outliers[:, 2], c='red', label='Outlier')

# Add legend and titles
ax.set_title('One-Class SVM Outlier Detection (3D)')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate (Path Length)')
ax.legend(loc='best')

plt.show()


#### k733_2018

##### optics

In [ ]:
from src.models.optics.optimize_optics_parallelized_no_aic_bic import optimize_optics_parallelized_no_aic_bic

optimization_results_k733_2018 = optimize_optics_parallelized_no_aic_bic(X=X_k733_2018, 
                                                                         max_eps_range=max_eps_range, 
                                                                         min_samples_range=min_samples_range, 
                                                                         metrics=metrics,
                                                                         cluster_methods=cluster_methods,
                                                                         xis=xis)
optimization_results_k733_2018

In [ ]:
from scipy import linalg
from sklearn.cluster import OPTICS
optics_optimized_silhouette_k733_2018 = OPTICS(min_samples=optimization_results_k733_2018['silhouette']['min_samples'],
                                               max_eps=optimization_results_k733_2018['silhouette']['epsilon'],
                                               metric=optimization_results_k733_2018['silhouette']['metric'],
                                               cluster_method=optimization_results_k733_2018['silhouette']['cluster_method']
                                               ).fit(X_k733_2018)
optics_optimized_silhouette_k733_2018

In [ ]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D  # Import the 3D plotting tools

labels_silhouette_k733_2018 = optics_optimized_silhouette_k733_2018.labels_
unique_labels = set(labels_silhouette_k733_2018)
colors = plt.cm.get_cmap('viridis', len(unique_labels)) 

fig_optics_k733_2018 = plt.figure()
ax = fig_optics_k733_2018.add_subplot(111, projection='3d')

# Plot each cluster with a unique color
for label in unique_labels:
    if label == -1:
        # Noise points (label == -1)
        color = 'gray'
    else:
        color = colors(label)  # Assign a color from the colormap

    # Select points belonging to the current label
    cluster_points = X_k733_2018[labels_silhouette_k733_2018 == label]

    # Plot the points in the cluster
    ax.plot(X_k733_2018[:,0], X_k733_2018[:,1], zs=0, zdir='z', color='gray', alpha=0.6)
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:,2], c=[color], label=f'Cluster {label}')

# Step 4: Add legend and titles
ax.set_title('OPTICS Clustering of Vehicle Paths')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.legend(loc='best')

plt.show()

##### one class svm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM
from mpl_toolkits.mplot3d import Axes3D  # Import the 3D plotting tools

# Step 2: Apply One-Class SVM for outlier detection
svm_model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)  # You can adjust the nu parameter for controlling the outlier ratio
svm_model.fit(X_k733_2020)

# Step 3: Get the labels (-1 for outliers, 1 for inliers)
labels = svm_model.predict(X_k733_2020)

In [ ]:
%matplotlib qt

fig_svm_k733_2020 = plt.figure()
ax = fig_svm_k733_2020.add_subplot(111, projection='3d')

# Use 'viridis' colormap for inliers and a specific color for outliers
colors = plt.cm.viridis(np.linspace(0, 1, len(X_k733_2020)))

# Plot background gray lines for all paths
ax.plot(X_k733_2020[:, 0], X_k733_2020[:, 1], zs=0, zdir='z', color='gray', alpha=0.6)

# Collect inliers and outliers for later plotting
inliers = X_k733_2020[labels == 1]   # Inliers
outliers = X_k733_2020[labels == -1] # Outliers

# Plot inliers (green)
ax.scatter(inliers[:, 0], inliers[:, 1], inliers[:, 2], c='green', label='Inlier')

# Plot outliers (red)
ax.scatter(outliers[:, 0], outliers[:, 1], outliers[:, 2], c='red', label='Outlier')

# Add legend and titles
ax.set_title('One-Class SVM Outlier Detection (3D)')
ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate (Path Length)')
ax.legend(loc='best')

plt.show()
